In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
from io import StringIO
import time
from datetime import datetime, timedelta
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
import re

import warnings
# Suppress FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

# Setup

### Variables modificables

In [ ]:
path_archivo_excel = "output.xlsx" # especificar el nombre del archivo de excel que se va a crear con la información
meses_filtro = 12 # especificar el número de meses máximos 

### Crear output.xlsx si no existe

In [ ]:
import os
from openpyxl import Workbook

# Verifica si el archivo ya existe
if not os.path.isfile(path_archivo_excel):
    # Crea un nuevo libro de trabajo (Workbook)
    workbook = Workbook()

    # Guarda el libro de trabajo en el archivo especificado
    workbook.save(path_archivo_excel)

    print(f'Se ha creado el archivo Excel: {path_archivo_excel}')
else:
    print(f'El archivo Excel ya existe: {path_archivo_excel}')

# Navegar por la página

Abrir página y mostrar el proceso en Firefox. Se recomienda usar este para poder asegurarse de estar en la página correcta. Esta sí Andi

In [ ]:
browser = webdriver.Firefox()
# browser = webdriver.Chrome()
browser.get("https://oficinajudicialvirtual.pjud.cl/home/index.php")

Ir a Inicio de Sesión

In [ ]:
boton_todos_los_servicios = browser.find_element(by='xpath', value="//button[text()='Todos los servicios']")
boton_todos_los_servicios.click()

boton_clave_unica = browser.find_element(by='xpath', value="//a[text()='Clave Única']")
boton_clave_unica.click()

Ingresar

In [ ]:
rut_usuario = '19.687.891-2'
clave_unica = 'Sofi2101!'

input_run = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="uname"]')))
input_run.send_keys(rut_usuario)  

input_clave_unica = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="pword"]')))
input_clave_unica.send_keys(clave_unica)  

boton_ingresar =  WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="login-submit"]')))
boton_ingresar.click()

Ir a Consulta Unificada

In [ ]:
sidebar = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="sidebar"]')))
consulta_unificada = WebDriverWait(sidebar, 10).until(EC.presence_of_element_located((By.XPATH, ".//a[contains(@onclick, 'consultaUnificada')]")))
consulta_unificada.click()

Leer archivo RUT´s bajadas.xlsx y mostrar los Ruts que serán buscados

In [ ]:
file_path = r'RUT´s bajadas 2.xlsx'

df = pd.read_excel(file_path)

ruts = df['RUT'].dropna().to_list() # [0:2]
print(ruts)
personas_juridicas = df['Buscar por nombre/persona jurídica'].dropna().to_list()
personas_naturales = df['Buscar por nombre/persona natural'].dropna().to_list()

### Búsqueda por Rut Persona Jurídica

Ir a Tab Búsqueda por Rut Persona Jurídica

In [ ]:
boton_busqueda_por_rut_persona_juridica = browser.find_element(by='xpath', value="//a[text()='Búsqueda por Rut Persona Jurídica']")
boton_busqueda_por_rut_persona_juridica.click()

Iterar por los ruts. En cada iteración se crea un df nuevo que contendrá toda la información de las tablas de todas las páginas que correspondan al rut, se lee el output.xlsx, se concatena la información, es decir a la información que ya se encuentra en el output.xlsx se le agrega la nueva información, luego se guarda la información en output.xlsx y finalmente se limpia la búsqueda en la página.

In [ ]:
for rut in ruts:
    success = False
    while not success:
        try:
            df = pd.DataFrame()
            no_registros = False
            print(f"rut: {rut}")
            rut1, rut2 = rut.split("-")  # separar rut en antes y después del "-", ya que hay inputs diferentes para cada uno. Por ejemplo 62920000-2: rut1=62920000, rut2=2
            
            input_rut = WebDriverWait(browser, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="rutJur"]')))
            input_rut.clear()  # Clear the input field
            input_rut.send_keys(rut1)
            
            input_rut2 = WebDriverWait(browser, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="dvJur"]')))
            input_rut2.clear()  # Clear the input field
            input_rut2.send_keys(rut2)
            
            boton_buscar = WebDriverWait(browser, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="btnConConsultaJur"]')))
            boton_buscar.click()
            time.sleep(5)
            
            while True:
                print(f"cantidad de filas recuperadas: {df.shape[0]}")
                try:
                    table_element = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="dtaTableDetalleJuridica"]')))
                    html = browser.page_source

                    # Use pandas to read the HTML content
                    df_list = pd.read_html(html)
                    df2 = pd.concat(df_list, ignore_index=True)
                    df2 = df2.iloc[4:-1, 4:]
                    df = pd.concat([df, df2])
                    time.sleep(5)

                except Exception as e:
                    print("No hay registros")
                    no_registros = True
                    boton_limpiar = WebDriverWait(browser, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="btnConLimpiarJur"]')))
                    boton_limpiar.click()
                    print(e)
                    break
                
                try:
                    boton_siguiente = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="sigId"]')))
                    boton_siguiente.click()
                    time.sleep(5)
                except Exception:
                    print("última página")
                    total_de_registros = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.loadTotalJur > b:nth-child(1)')))
                    total_de_registros = total_de_registros.get_attribute('outerHTML')
                    total_de_registros = re.sub(r'<.*?>', '', total_de_registros)
                    print(f"total de registros: {total_de_registros}")
                    break

            print(f"cantidad final de filas recuperadas: {df.shape[0]}")

            # Check if the total number of records matches the rows retrieved
            if not no_registros and int(total_de_registros) != df.shape[0]:
                print("Mismatch in total de registros and cantidad de filas recuperadas, retrying...")
                boton_limpiar = WebDriverWait(browser, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="btnConLimpiarJur"]')))
                boton_limpiar.click()
                time.sleep(5)
                continue  # Retry the same rut

            # Filter by date if records are found
            if not no_registros:
                df['Parsed Fecha Ingreso'] = pd.to_datetime(df['Fecha Ingreso'], format='%d/%m/%Y')
                n_months_ago = datetime.now() - timedelta(days=meses_filtro * 30)
                date_condition = (df['Parsed Fecha Ingreso'] >= n_months_ago)
                df = df[date_condition]
                df = df.drop(columns=['Parsed Fecha Ingreso'])
                print(f"cantidad de filas después del filtro (12 meses): {df.shape[0]}")

                # Combine with existing data and save
                df_existing = pd.read_excel(f"{path_archivo_excel}")
                df_combined = pd.concat([df_existing, df], ignore_index=True)
                df_combined.to_excel(f"{path_archivo_excel}", index=False)

            # Clean up for the next iteration
            boton_limpiar = WebDriverWait(browser, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="btnConLimpiarJur"]')))
            boton_limpiar.click()
            time.sleep(5)
            success = True  # Exit the retry loop

        except Exception as e:
            print(f"Exception occurred for rut {rut}: {e}")
            # Optionally, add some handling logic or continue to the next rut
            break


### Búsqueda por Nombre

In [ ]:
# boton_busqueda_por_nombre = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div/div[2]/div[2]/div[1]/div/section/div[1]/div/ul/li[2]/a")))
boton_busqueda_por_nombre = browser.find_element(by='xpath', value="//a[text()='Búsqueda por Nombre']")
boton_busqueda_por_nombre.click()

#### Persona Natural

In [ ]:
boton_persona_natural = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="radioPerNatural"]')))
boton_persona_natural.click()

In [ ]:
for persona_natural in personas_naturales:
    df = pd.DataFrame()
    no_registros = False
    nombre, apellido_paterno, apellido_materno = persona_natural.split()
    print(f"nombre: {nombre} {apellido_paterno} {apellido_materno}")

    input_nombre = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="nomNombre"]')))
    input_nombre.send_keys(nombre)
    input_apellido_paterno = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="nomApePaterno"]')))
    input_apellido_paterno.send_keys(apellido_paterno)
    input_apellido_materno = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="nomApeMaterno"]')))
    input_apellido_materno.send_keys(apellido_materno)

    boton_buscar = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="btnConConsultaNom"]')))
    boton_buscar.click()
    time.sleep(5)


    while True:
        print(f"cantidad de filas recuperadas: {df.shape[0]}")
        try:
            table_element = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="dtaTableDetalleNombre"]')))
            html = browser.page_source
            # Use pandas to read the HTML content
            df_list = pd.read_html(html)
            df2 = pd.concat(df_list, ignore_index=True)
            df2 = df2.iloc[2:-3, 4:]
            df = pd.concat([df,df2])
            time.sleep(5)
            display(df)

        except Exception as e:
            print("No hay registros")
            no_registros = True
            boton_limpiar = WebDriverWait(browser, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="btnConLimpiarNom"]')))
            boton_limpiar.click()
            print(e)
            break
        
        try:
            boton_siguiente = WebDriverWait(browser, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="sigId"]')))
            boton_siguiente.click()
            time.sleep(5)
            
        except Exception as e:
            print("última página")
            if df.shape[0] == 0:
                print('No hay registros')
                no_registros = True
                break

            total_de_registros = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.loadTotalNom > b:nth-child(1)')))
            total_de_registros = total_de_registros.get_attribute('outerHTML')
            total_de_registros = re.sub(r'<.*?>', '', total_de_registros)
            print(f"total de registros: {total_de_registros}")
            break

    print(f"cantidad final de filas recuperadas: {df.shape[0]}")

    if no_registros:
        no_registros = False
    else:    
        # filtrar por fecha
        df['Parsed Fecha Ingreso'] = pd.to_datetime(df['Fecha Ingreso'], format='%d/%m/%Y')
        n_months_ago = datetime.now() - timedelta(days=meses_filtro*30)
        date_condition = (df['Parsed Fecha Ingreso'] >= n_months_ago)
        df = df[date_condition]
        df = df.drop(columns=['Parsed Fecha Ingreso'])
        print(f"cantidad de filas después del filtro (12 meses): {df.shape[0]}")
        
        df_existing = pd.read_excel(f"{path_archivo_excel}")

        df_combined = pd.concat([df_existing, df], ignore_index=True)
        df_combined.to_excel(f"{path_archivo_excel}", index=False)    

    boton_limpiar = WebDriverWait(browser, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="btnConLimpiarNom"]')))
    boton_limpiar.click()
    time.sleep(5)    

# Lo que sigue todavía no ha sido arreglado ni testeado

#### Persona Jurídica

In [ ]:
boton_persona_juridica = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="radioPerJuridica"]')))
boton_persona_juridica.click()

In [ ]:
for persona_juridica in personas_juridicas:
    df = pd.DataFrame()
    no_registros = False
    print(persona_juridica)
    input_nombre = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="nomNombreJur"]')))
    input_nombre.send_keys(persona_juridica)
    boton_buscar = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="btnConConsultaNom"]')))
    boton_buscar.click()
    time.sleep(0.5)

    while True:
        print(f"cantidad de filas recuperadas: {df.shape[0]}")
        try:
            table_element = WebDriverWait(browser, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="dtaTableDetalleNombre"]')))
            html_content = table_element.get_attribute('outerHTML')
            df2 = pd.read_html(StringIO(html_content), displayed_only=False)[0].iloc[:-1]
            df = pd.concat([df,df2])
            time.sleep(1)

        except Exception as e:
            print("No hay registros")
            no_registros = True
            boton_limpiar = WebDriverWait(browser, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="btnConLimpiarNom"]')))
            boton_limpiar.click()
            # print(e)
            break
        
        time.sleep(1)
        try:
            boton_siguiente = WebDriverWait(browser, 4).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="sigId"]')))
            boton_siguiente.click()
            time.sleep(1)
        except Exception as e:
            print("última página")
            # print(e)
            break
    print(f"cantidad final de filas recuperadas: {df.shape[0]}")

    if no_registros:
        continue
    
    # filtrar por fecha
    df['Parsed Fecha Ingreso'] = pd.to_datetime(df['Fecha Ingreso'], format='%d/%m/%Y')
    n_months_ago = datetime.now() - timedelta(days=meses_filtro*30)
    date_condition = (df['Parsed Fecha Ingreso'] >= n_months_ago)
    df = df[date_condition]
    df = df.drop(columns=['Parsed Fecha Ingreso'])
    #
    
    df_existing = pd.read_excel(f"{path_archivo_excel}")
    df_combined = pd.concat([df_existing, df], ignore_index=True)
    df_combined.to_excel(f"{path_archivo_excel}", index=False)    
    boton_limpiar = WebDriverWait(browser, 4).until(EC.presence_of_element_located((By.XPATH, '//*[@id="btnConLimpiarNom"]')))
    boton_limpiar.click()
    time.sleep(1)    

In [ ]:
import pandas as pd

url = "https://oficinajudicialvirtual.pjud.cl/indexN.php"
dfs = pd.read_html(url)

print(len(dfs))